<a href="https://colab.research.google.com/github/BatsaikhanTuguldur/KousenProjects/blob/main/INCT_ML11_Dogs_vs_Cats_Prediction_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 8: Dogs vs Cats - Prediction
※このファイルは香川高専（詫間キャンパス）で開催されたディープラーニング初級講座の内容を基に作成しました。

Chapter 7で保存したDogs vs Catsの学習済みモデルを使って，任意の画像データに対してイヌとネコの分類をしてみましょう。

### ライブラリのインポート
必要なライブラリをインポートしましょう。

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
np.set_printoptions(precision=3, suppress=True)

from tqdm import tqdm
import os
import random
import cv2

# バージョン指定インストール
# h5 ファイルの読み込みにバージョン合わせが必要になったらしい。
#https://qiita.com/Hiroki-Fujimoto/items/b078bfb680fb710c38c1
!sudo pip3 install h5py==2.10.0

# Google Colab上のTensorFlowは，もうすぐバージョン1.15から2.0に
# アップグレードされるらしいので，引き続き1.15を使用したい場合は，
# 次のマジックコマンドを使う。
%tensorflow_version 1.x

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from tensorflow.python.keras import backend as K

# 念のためTensorFlowのバージョンを確認
# 1.15.0と表示されればOK
print("TensorFlow version: " + tf.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.1/301.1 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 417, in run
    _, build_failures = build(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 320, in build
    wheel_file = _build_one(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 194, in _build_one
    wheel_path = _build_one_inside_env(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 241, in _build_one_inside_env
    wheel_path = build_wheel_legac

ValueError: ignored

### Googleドライブのマウント
いつものようにGoogleドライブをマウントし，ファイルにアクセスできるようにしてください。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 学習済みモデルの読み込み
Chapter 7で保存した学習済みモデル```dogs_vs_cats_model.h5```を読み込みます。

In [ ]:
# 学習済みモデルのパスを指定
path = '/content/drive/MyDrive/智治先生の授業.h5'

# モデルを読み込み
#model = load_model(path)
model = tf.keras.models.load_model(path)

# 読み込んだモデルを可視化して確認
plot_model(model, show_shapes=True)

別の方法でも読み込んだモデルを確認しておきましょう。

In [ ]:
# モデルを別の方法で可視化して確認
model.summary()

### 判別したい画像の読み込み
イヌ・ネコの判別をしたい任意の画像を読み込みます。

In [ ]:
# 画像ファイルのパスを指定
path = "/content/drive/MyDrive/智治先生の授業/dog_1.jpg"
#path = "/content/drive/My Drive/INCT-ML/cat.jpg"
filename = os.path.basename(path)

# 画像の読み込みと表示
img = cv2.imread(path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.axis("off")
title = "{}\n{}".format(filename, img.shape)
plt.title(title)

### 読み込んだ画像のフォーマットを変更
学習済みモデルに入力できるように読み込んだ画像のサイズ等を変更します。

In [ ]:
# モデルに合うようにnewsizeを設定
# Chapter 7でtrainXを生成したときに自分が設定したサイズと同じ
# である必要がある。もう一度手動で指定しても良いが，次のコードなら
# 読み込んだモデルから適切なサイズを自動的に設定できる
newsize = model.layers[0].input_shape[1:3]
print("newsize=", newsize)

# 画像サイズを変更
img_modified = cv2.resize(img, newsize)

# ピクセル値の最大値が1.0になるように規格化
img_modified = img_modified/255.0

# 表示して確認
plt.imshow(img_modified)
plt.title(img_modified.shape)
plt.axis("off")

### 入力データの生成
モデルの入力サイズに合うように入力データの次元を変更します。

In [ ]:
input_data = np.expand_dims(img_modified, 0)
input_data.shape

### 推論
入力データに対してイヌかネコか判別します。

In [ ]:
pred = model.predict(input_data)
score = pred[0]

print("cat={:.2f}, dog={:.2f}".format(score[0], score[1]))

## ヒートマップ
ヒートマップ（正確にはGrad-CAM)と言う技術を使うと，モデルがどこをみてイヌ・ネコを判断したのかを知ることができます。詳しい説明はしませんが，とりあえず使ってみましょう。

* **Grad-CAM** https://arxiv.org/pdf/1610.02391.pdf

モデルの中で一番最後にある畳み込み層(Conv2D)の名前を指定してください。

※最後の畳み込み層の名前は個人ごとに違います！

In [ ]:
# モデルを可視化して確認
model.summary()

In [ ]:
# 最後のconvalutional layerを指定
last_conv_layer = model.get_layer('conv2d_23')

次のセルを実行するとヒートマップが生成されます。

※ヒートマップ作成のプログラムを理解する必要はありません。出力されたヒートマップが何を示しているのかを理解してください。

In [ ]:
N = np.argmax(pred)
model_output = model.output[:,N]

grads = K.gradients(model_output, last_conv_layer.output)[0]
pooled_grads = K.mean(grads, axis =(0,1,2))

iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])

pooled_grads_val, conv_output_val = iterate([input_data])

print(conv_output_val.shape)     # (15, 15, 128)
print(pooled_grads_val.shape)    # (128, )

for i in range(pooled_grads_val.shape[0]):
    conv_output_val[:, :, i] *= pooled_grads_val[i]



# ヒートマップの生成
heatmap = np.mean(conv_output_val, axis = -1)

# ヒートマップの後処理
heatmap = np.maximum(heatmap, 0)
heatmap = heatmap/heatmap.max()

# 元の画像と同じサイズになるようにヒートマップのサイズを変更
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

# ヒートマップをRGBに変換
heatmap = np.uint8(255 * heatmap)
# JETのカラーマップを適用
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
# RGBに変換
heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

plt.imshow(heatmap, 'jet')
plt.colorbar()



ヒートマップを元の画像に重ねて表示してみましょう。

In [ ]:
# ヒートマップと元の画像を重ねる
# 0.4はヒートマップの強度係数
superimposed_img = heatmap*0.4 + img
superimposed_img = superimposed_img/np.max(superimposed_img)

# 画像を表示
plt.imshow(superimposed_img)